# HAR CNN training 

In [1]:
# Imports
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
%ls data/train/Inertial_Signals/

body_acc_x_train.txt*  body_gyro_x_train.txt*  total_acc_x_train.txt*
body_acc_y_train.txt*  body_gyro_y_train.txt*  total_acc_y_train.txt*
body_acc_z_train.txt*  body_gyro_z_train.txt*  total_acc_z_train.txt*


In [3]:
X_train, labels_train, list_ch_train = read_data(data_path="./data/", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path="./data/", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

In [4]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [5]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [6]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Hyperparameters

In [8]:
batch_size = 600       # Batch size
seq_len = 128          # Number of steps
learning_rate = 0.0001
epochs = 1000

n_classes = 6
n_channels = 9

### Construct the graph
Placeholders

In [9]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

Note: Should we use a different activation? Like tf.nn.tanh?

In [10]:
with graph.as_default():
    # (batch, 128, 9) --> (batch, 64, 18)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Now, flatten and pass to the classifier

In [11]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_4, (-1, 8*144))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [12]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 1.841065 Train acc: 0.178333
Epoch: 1/1000 Iteration: 10 Train loss: 1.793845 Train acc: 0.208333
Epoch: 1/1000 Iteration: 10 Validation loss: 1.761763 Validation acc: 0.237778
Epoch: 1/1000 Iteration: 15 Train loss: 1.741301 Train acc: 0.245000
Epoch: 2/1000 Iteration: 20 Train loss: 1.691000 Train acc: 0.250000
Epoch: 2/1000 Iteration: 20 Validation loss: 1.676826 Validation acc: 0.319444
Epoch: 2/1000 Iteration: 25 Train loss: 1.656695 Train acc: 0.263333
Epoch: 3/1000 Iteration: 30 Train loss: 1.625954 Train acc: 0.290000
Epoch: 3/1000 Iteration: 30 Validation loss: 1.613114 Validation acc: 0.322222
Epoch: 3/1000 Iteration: 35 Train loss: 1.599463 Train acc: 0.293333
Epoch: 4/1000 Iteration: 40 Train loss: 1.577778 Train acc: 0.296667
Epoch: 4/1000 Iteration: 40 Validation loss: 1.558642 Validation acc: 0.339444
Epoch: 4/1000 Iteration: 45 Train loss: 1.573984 Train acc: 0.270000
Epoch: 5/1000 Iteration: 50 Train loss: 1.519228 Train acc: 0.36

Epoch: 42/1000 Iteration: 385 Train loss: 0.209662 Train acc: 0.926667
Epoch: 43/1000 Iteration: 390 Train loss: 0.247150 Train acc: 0.908333
Epoch: 43/1000 Iteration: 390 Validation loss: 0.200616 Validation acc: 0.922222
Epoch: 43/1000 Iteration: 395 Train loss: 0.201644 Train acc: 0.928333
Epoch: 44/1000 Iteration: 400 Train loss: 0.228571 Train acc: 0.913333
Epoch: 44/1000 Iteration: 400 Validation loss: 0.194613 Validation acc: 0.923889
Epoch: 44/1000 Iteration: 405 Train loss: 0.228127 Train acc: 0.926667
Epoch: 45/1000 Iteration: 410 Train loss: 0.215479 Train acc: 0.920000
Epoch: 45/1000 Iteration: 410 Validation loss: 0.189324 Validation acc: 0.925000
Epoch: 46/1000 Iteration: 415 Train loss: 0.216057 Train acc: 0.921667
Epoch: 46/1000 Iteration: 420 Train loss: 0.184980 Train acc: 0.926667
Epoch: 46/1000 Iteration: 420 Validation loss: 0.184366 Validation acc: 0.927778
Epoch: 47/1000 Iteration: 425 Train loss: 0.190102 Train acc: 0.945000
Epoch: 47/1000 Iteration: 430 Train l

Epoch: 84/1000 Iteration: 765 Train loss: 0.151316 Train acc: 0.948333
Epoch: 85/1000 Iteration: 770 Train loss: 0.132657 Train acc: 0.955000
Epoch: 85/1000 Iteration: 770 Validation loss: 0.120726 Validation acc: 0.948333
Epoch: 86/1000 Iteration: 775 Train loss: 0.125518 Train acc: 0.946667
Epoch: 86/1000 Iteration: 780 Train loss: 0.112869 Train acc: 0.950000
Epoch: 86/1000 Iteration: 780 Validation loss: 0.119738 Validation acc: 0.948889
Epoch: 87/1000 Iteration: 785 Train loss: 0.130508 Train acc: 0.951667
Epoch: 87/1000 Iteration: 790 Train loss: 0.141909 Train acc: 0.943333
Epoch: 87/1000 Iteration: 790 Validation loss: 0.119031 Validation acc: 0.947778
Epoch: 88/1000 Iteration: 795 Train loss: 0.147229 Train acc: 0.928333
Epoch: 88/1000 Iteration: 800 Train loss: 0.098393 Train acc: 0.963333
Epoch: 88/1000 Iteration: 800 Validation loss: 0.118330 Validation acc: 0.948333
Epoch: 89/1000 Iteration: 805 Train loss: 0.145955 Train acc: 0.926667
Epoch: 89/1000 Iteration: 810 Train l

Epoch: 126/1000 Iteration: 1135 Train loss: 0.105715 Train acc: 0.960000
Epoch: 126/1000 Iteration: 1140 Train loss: 0.091192 Train acc: 0.961667
Epoch: 126/1000 Iteration: 1140 Validation loss: 0.105636 Validation acc: 0.954444
Epoch: 127/1000 Iteration: 1145 Train loss: 0.103328 Train acc: 0.965000
Epoch: 127/1000 Iteration: 1150 Train loss: 0.116289 Train acc: 0.948333
Epoch: 127/1000 Iteration: 1150 Validation loss: 0.105392 Validation acc: 0.954444
Epoch: 128/1000 Iteration: 1155 Train loss: 0.124694 Train acc: 0.938333
Epoch: 128/1000 Iteration: 1160 Train loss: 0.085941 Train acc: 0.970000
Epoch: 128/1000 Iteration: 1160 Validation loss: 0.105116 Validation acc: 0.954444
Epoch: 129/1000 Iteration: 1165 Train loss: 0.128366 Train acc: 0.940000
Epoch: 129/1000 Iteration: 1170 Train loss: 0.120795 Train acc: 0.956667
Epoch: 129/1000 Iteration: 1170 Validation loss: 0.104890 Validation acc: 0.954444
Epoch: 130/1000 Iteration: 1175 Train loss: 0.110433 Train acc: 0.961667
Epoch: 131/

Epoch: 166/1000 Iteration: 1495 Train loss: 0.089989 Train acc: 0.965000
Epoch: 166/1000 Iteration: 1500 Train loss: 0.087412 Train acc: 0.961667
Epoch: 166/1000 Iteration: 1500 Validation loss: 0.098314 Validation acc: 0.958889
Epoch: 167/1000 Iteration: 1505 Train loss: 0.100906 Train acc: 0.960000
Epoch: 167/1000 Iteration: 1510 Train loss: 0.105077 Train acc: 0.958333
Epoch: 167/1000 Iteration: 1510 Validation loss: 0.098086 Validation acc: 0.959444
Epoch: 168/1000 Iteration: 1515 Train loss: 0.123748 Train acc: 0.940000
Epoch: 168/1000 Iteration: 1520 Train loss: 0.072377 Train acc: 0.970000
Epoch: 168/1000 Iteration: 1520 Validation loss: 0.097885 Validation acc: 0.958889
Epoch: 169/1000 Iteration: 1525 Train loss: 0.126505 Train acc: 0.940000
Epoch: 169/1000 Iteration: 1530 Train loss: 0.116897 Train acc: 0.958333
Epoch: 169/1000 Iteration: 1530 Validation loss: 0.097794 Validation acc: 0.958889
Epoch: 170/1000 Iteration: 1535 Train loss: 0.105479 Train acc: 0.960000
Epoch: 171/

...

Epoch: 571/1000 Iteration: 5140 Train loss: 0.022286 Train acc: 0.991667
Epoch: 571/1000 Iteration: 5140 Validation loss: 0.048953 Validation acc: 0.982778
Epoch: 571/1000 Iteration: 5145 Train loss: 0.022777 Train acc: 0.988333
Epoch: 572/1000 Iteration: 5150 Train loss: 0.018247 Train acc: 0.995000
Epoch: 572/1000 Iteration: 5150 Validation loss: 0.049039 Validation acc: 0.981111
Epoch: 572/1000 Iteration: 5155 Train loss: 0.022516 Train acc: 0.990000
Epoch: 573/1000 Iteration: 5160 Train loss: 0.031469 Train acc: 0.990000
Epoch: 573/1000 Iteration: 5160 Validation loss: 0.048769 Validation acc: 0.983333
Epoch: 573/1000 Iteration: 5165 Train loss: 0.018005 Train acc: 0.993333
Epoch: 574/1000 Iteration: 5170 Train loss: 0.028294 Train acc: 0.993333
Epoch: 574/1000 Iteration: 5170 Validation loss: 0.049066 Validation acc: 0.983333
Epoch: 574/1000 Iteration: 5175 Train loss: 0.022944 Train acc: 0.988333
Epoch: 575/1000 Iteration: 5180 Train loss: 0.023720 Train acc: 0.993333
Epoch: 575/

Epoch: 612/1000 Iteration: 5510 Validation loss: 0.048915 Validation acc: 0.982778
Epoch: 612/1000 Iteration: 5515 Train loss: 0.021359 Train acc: 0.991667
Epoch: 613/1000 Iteration: 5520 Train loss: 0.028770 Train acc: 0.986667
Epoch: 613/1000 Iteration: 5520 Validation loss: 0.049492 Validation acc: 0.984444
Epoch: 613/1000 Iteration: 5525 Train loss: 0.011739 Train acc: 0.995000
Epoch: 614/1000 Iteration: 5530 Train loss: 0.022999 Train acc: 0.991667
Epoch: 614/1000 Iteration: 5530 Validation loss: 0.048774 Validation acc: 0.983333
Epoch: 614/1000 Iteration: 5535 Train loss: 0.023303 Train acc: 0.990000
Epoch: 615/1000 Iteration: 5540 Train loss: 0.023309 Train acc: 0.991667
Epoch: 615/1000 Iteration: 5540 Validation loss: 0.048871 Validation acc: 0.984444
Epoch: 616/1000 Iteration: 5545 Train loss: 0.024309 Train acc: 0.990000
Epoch: 616/1000 Iteration: 5550 Train loss: 0.019891 Train acc: 0.991667
Epoch: 616/1000 Iteration: 5550 Validation loss: 0.049023 Validation acc: 0.985556
E

Epoch: 652/1000 Iteration: 5870 Validation loss: 0.049146 Validation acc: 0.983889
Epoch: 652/1000 Iteration: 5875 Train loss: 0.017745 Train acc: 0.991667
Epoch: 653/1000 Iteration: 5880 Train loss: 0.024357 Train acc: 0.993333
Epoch: 653/1000 Iteration: 5880 Validation loss: 0.049450 Validation acc: 0.985000
Epoch: 653/1000 Iteration: 5885 Train loss: 0.011028 Train acc: 0.996667
Epoch: 654/1000 Iteration: 5890 Train loss: 0.020222 Train acc: 0.995000
Epoch: 654/1000 Iteration: 5890 Validation loss: 0.050285 Validation acc: 0.984444
Epoch: 654/1000 Iteration: 5895 Train loss: 0.016928 Train acc: 0.991667
Epoch: 655/1000 Iteration: 5900 Train loss: 0.018061 Train acc: 0.991667
Epoch: 655/1000 Iteration: 5900 Validation loss: 0.049760 Validation acc: 0.983889
Epoch: 656/1000 Iteration: 5905 Train loss: 0.020929 Train acc: 0.991667
Epoch: 656/1000 Iteration: 5910 Train loss: 0.015726 Train acc: 0.993333
Epoch: 656/1000 Iteration: 5910 Validation loss: 0.048697 Validation acc: 0.984444
E

Epoch: 692/1000 Iteration: 6235 Train loss: 0.013036 Train acc: 0.996667
Epoch: 693/1000 Iteration: 6240 Train loss: 0.022176 Train acc: 0.993333
Epoch: 693/1000 Iteration: 6240 Validation loss: 0.049657 Validation acc: 0.985000
Epoch: 693/1000 Iteration: 6245 Train loss: 0.010938 Train acc: 0.996667
Epoch: 694/1000 Iteration: 6250 Train loss: 0.017130 Train acc: 0.995000
Epoch: 694/1000 Iteration: 6250 Validation loss: 0.050248 Validation acc: 0.984444
Epoch: 694/1000 Iteration: 6255 Train loss: 0.014071 Train acc: 0.996667
Epoch: 695/1000 Iteration: 6260 Train loss: 0.019591 Train acc: 0.991667
Epoch: 695/1000 Iteration: 6260 Validation loss: 0.050082 Validation acc: 0.983889
Epoch: 696/1000 Iteration: 6265 Train loss: 0.016132 Train acc: 0.996667
Epoch: 696/1000 Iteration: 6270 Train loss: 0.011699 Train acc: 0.996667
Epoch: 696/1000 Iteration: 6270 Validation loss: 0.049444 Validation acc: 0.986111
Epoch: 697/1000 Iteration: 6275 Train loss: 0.009678 Train acc: 0.998333
Epoch: 697/

Epoch: 733/1000 Iteration: 6600 Train loss: 0.017876 Train acc: 0.995000
Epoch: 733/1000 Iteration: 6600 Validation loss: 0.050013 Validation acc: 0.985556
Epoch: 733/1000 Iteration: 6605 Train loss: 0.009776 Train acc: 0.996667
Epoch: 734/1000 Iteration: 6610 Train loss: 0.017971 Train acc: 0.995000
Epoch: 734/1000 Iteration: 6610 Validation loss: 0.051259 Validation acc: 0.986111
Epoch: 734/1000 Iteration: 6615 Train loss: 0.012377 Train acc: 0.998333
Epoch: 735/1000 Iteration: 6620 Train loss: 0.012680 Train acc: 0.996667
Epoch: 735/1000 Iteration: 6620 Validation loss: 0.051056 Validation acc: 0.986111
Epoch: 736/1000 Iteration: 6625 Train loss: 0.014719 Train acc: 0.995000
Epoch: 736/1000 Iteration: 6630 Train loss: 0.013886 Train acc: 0.993333
Epoch: 736/1000 Iteration: 6630 Validation loss: 0.050271 Validation acc: 0.985556
Epoch: 737/1000 Iteration: 6635 Train loss: 0.010009 Train acc: 0.996667
Epoch: 737/1000 Iteration: 6640 Train loss: 0.014348 Train acc: 0.995000
Epoch: 737/

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [20]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints-cnn/har.ckpt
Test accuracy: 0.913750
